In [ ]:
import time

import torch
import numpy as np
import pandas as pd
from torch import nn
from torchtext.datasets import AG_NEWS
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer
from torch.utils.data.dataset import random_split
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.functional import to_map_style_dataset
from giskard import Model, Dataset, GiskardClient, scan, testing

## Define constants

In [ ]:
DEVICE = torch.device("cpu")

TARGET_MAP = {1: "World", 2: "Sports", 3: "Business", 4: "Sci/Tech"}
TARGET_COLUMN_NAME = "label"
FEATURE_COLUMN_NAME = "text"

LOADERS_BATCH_SIZE = 64

## Dataset preparation

### Load data

In [ ]:
train_data, test_data = AG_NEWS()

### Wrap dataset with Giskard

In [ ]:
raw_data = pd.DataFrame({FEATURE_COLUMN_NAME: text, TARGET_COLUMN_NAME: TARGET_MAP[label_id]} for label_id, text in test_data)
wrapped_data = Dataset(raw_data, name="Test Dataset", target="label", feature_types={FEATURE_COLUMN_NAME: "text", TARGET_COLUMN_NAME: "category"})

### Prepare dataloaders for training and evaluation

In [ ]:
# Simple English tokenizer provided by torchtext.
tokenizer = get_tokenizer("basic_english")

# Build a vocabulary from all the tokens we can find in the train data.
vocab = build_vocab_from_iterator((tokenizer(text) for _, text in train_data), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])


def preprocess_text(raw_text):
    return vocab(tokenizer(raw_text))


def preprocess_label(raw_label):
    return int(raw_label) - 1


def collate_fn(batch):
    label_list, text_list, offsets = [], [], [0]

    for _label, _text in batch:
        label_list.append(preprocess_label(_label))
        processed_text = torch.tensor(preprocess_text(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))

    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)

    return label_list.to(DEVICE), text_list.to(DEVICE), offsets.to(DEVICE)


# Create the datasets
train_dataset = to_map_style_dataset(train_data)
test_dataset = to_map_style_dataset(test_data)

# We further divide the training data into a train and validation split.
train_split, valid_split = random_split(train_dataset, [0.95, 0.05])

# Prepare the data loaders
train_dataloader = DataLoader(train_split, batch_size=LOADERS_BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
valid_dataloader = DataLoader(valid_split, batch_size=LOADERS_BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=LOADERS_BATCH_SIZE, shuffle=True, collate_fn=collate_fn)

## Model training

### Define model

In [ ]:
class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        init_range = 0.5
        self.embedding.weight.data.uniform_(-init_range, init_range)
        self.fc.weight.data.uniform_(-init_range, init_range)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded).softmax(axis=-1)

model = TextClassificationModel(vocab_size=vocab, embed_dim=64, num_class=4).to(DEVICE)

### Train and evaluate model

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.1)


def train_epoch(dataloader):
    model.train()

    train_accuracy = total_count = 0
    for label, text, offset in dataloader:
        optimizer.zero_grad()
        predicted_label = model(text, offset)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        train_accuracy += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)

    return train_accuracy / total_count


def validation_epoch(dataloader):
    model.eval()

    validation_accuracy = total_count = 0
    with torch.no_grad():
        for label, text, offsets in dataloader:
            predicted_label = model(text, offsets)
            validation_accuracy += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)

    return validation_accuracy / total_count


total_accuracy = None
for epoch in range(1, 11):
    start_time = time.perf_counter()

    train_epoch(train_dataloader)
    accu_val = validation_epoch(valid_dataloader)

    if total_accuracy is not None and total_accuracy > accu_val:
        scheduler.step()
    else:
        total_accuracy = accu_val

    print("-" * 59)
    print(f"| end of epoch {epoch: .3f} | time: {time.perf_counter() - start_time :5.2f}s | valid accuracy {accu_val:8.3f} ")
    print("-" * 59)

In [ ]:
test_accuracy = validation_epoch(test_dataloader)
print('Test accuracy {:8.3f}'.format(test_accuracy))

### Infer predictions on the example data

In [ ]:
def predict(text):
    with torch.no_grad():
        text = torch.tensor(preprocess_text(text))
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() + 1

In [ ]:
example_news = (
    "MEMPHIS, Tenn. – Four days ago, Jon Rahm was "
    "enduring the season’s worst weather conditions on Sunday at The "
    "Open on his way to a closing 75 at Royal Portrush, which "
    "considering the wind and the rain was a respectable showing. "
    "Thursday’s first round at the WGC-FedEx St. Jude Invitational "
    "was another story. With temperatures in the mid-80s and hardly any "
    "wind, the Spaniard was 13 strokes better in a flawless round. "
    "Thanks to his best putting performance on the PGA Tour, Rahm "
    "finished with an 8-under 62 for a three-stroke lead, which "
    "was even more impressive considering he’d never played the "
    "front nine at TPC Southwind."
)

predicted = predict(example_news)

### Wrap model with Giskard

In [ ]:
def prediction_function(df) -> np.ndarray:
    predictions = predict(df.text)
    predictions = predictions.cpu().detach().numpy()
    return predictions

In [ ]:
wrapped_model = Model(model,
                      name="SimpleNewsClassificationModel",
                      feature_names=["text"],
                      model_type="classification",
                      classification_labels=list(TARGET_MAP.values()))

# Validate wrapped model.
wrapped_model.predict(wrapped_data)

### Scan model with Giskard

In [ ]:
results = scan(wrapped_model, wrapped_data)

In [ ]:
display(results)

## Generate a test suite from the Scan
The objects produced by the scan can be used as fixtures to generate a test suite that integrate domain-specific issues. To create custom tests, refer to the Test your ML Model page.

In [ ]:
test_suite = results.generate_test_suite("My first test suite")
test_suite.run()

## Customize your suite by loading objects from the Giskard catalog

The Giskard open source catalog will enable to load:
* Tests such as metamorphic, performance, prediction & data drift, statistical tests, etc
* Slicing functions such as detectors of toxicity, hate, emotion, etc
* Transformation functions such as generators of typos, paraphrase, style tune, etc

For demo purposes, we will load a simple unit test (test_f1) that checks if the test F1 score is above the given threshold. For more examples of tests and functions, refer to the Giskard catalog.

In [ ]:
test_suite.add_test(testing.test_f1(model=wrapped_model, dataset=wrapped_data, threshold=0.7)).run()

## Upload your suite to the Giskard server

Upload your suite to the Giskard server to:
* Compare models to decide which model to promote
* Debug your tests to diagnose the issues
* Create more domain-specific tests that are integrating business feedback
* Share your results

In [ ]:
# Uploading the test suite will automatically save the model, dataset, tests, slicing & transformation functions inside the Giskard UI server
# Create a Giskard client after having install the Giskard server (see documentation)
token = "API_TOKEN"  # Find it in Settings in the Giskard server

client = GiskardClient(
    url="http://localhost:19000",  # URL of your Giskard instance
    token=token
)

my_project = client.create_project("my_project", "PROJECT_NAME", "DESCRIPTION")

# Upload to the current project ✉️
test_suite.upload(client, "my_project")